##### Step 1: Convert CSV to JSON Files

In [2]:
import csv
import json
import uuid
import os

raw_reviews_file = "../data/raw/hotel_reviews_1000.csv"
transformed_dir = "../data/transformed"

raw_reviews = open(raw_reviews_file, "r").readlines()

if not os.path.exists(transformed_dir):
    os.makedirs(transformed_dir)

def process_reviews(file_path):
    with open(file_path, 'r', newline='', encoding='utf-8') as csvfile:
        # Read the first line to get the header
        header = next(csv.reader(csvfile))
        
        # Create a mapping of expected column names to actual column names
        column_mapping = {
            'dateAdded': 'dateAdded',
            'city': 'city',
            'hotel_name': 'name',
            'hotel_state': 'province',
            'review_text': 'reviews.text',
            'review_title': 'reviews.title'
        }
        
        # Find the index of each required column
        column_indices = {}
        for expected_name, actual_name in column_mapping.items():
            try:
                column_indices[expected_name] = header.index(actual_name)
            except ValueError:
                print(f"Warning: Column '{actual_name}' not found in the CSV. Some data may be missing.")
        
        # Reset file pointer to the beginning
        csvfile.seek(0)
        
        # Skip the header row
        next(csvfile)
        
        # Use csv.reader instead of DictReader
        reader = csv.reader(csvfile)
        
        for i, row in enumerate(reader, start=1):
            review_json = {}
            for key, index in column_indices.items():
                if index < len(row):
                    review_json[key] = row[index]
                else:
                    review_json[key] = ""  # or None, depending on your preference
            
            # Generate a unique identifier
            review_json['id'] = str(uuid.uuid4())
            
            # print(json.dumps(review_json, indent=2))
            print(f"processed record [{i}] with id [{review_json['id']}]")

            with open(f"{transformed_dir}/review_{i}.json", "w+") as f:
                json.dump(review_json, f, indent=2)
            
process_reviews(raw_reviews_file)

processed record [1] with id [ba725625-9041-43b0-a4f6-698e812c2f87]
processed record [2] with id [087ce129-8e2c-4ee1-9ad8-1e5e286c6f30]
processed record [3] with id [2749eb2c-2a58-4c3e-9774-78d3d280cfa6]
processed record [4] with id [44b49048-27eb-49a9-a83c-06bbe1274fd1]
processed record [5] with id [a81d20b9-889c-4b35-bec2-501012703370]
processed record [6] with id [1ab48123-198d-4de3-b7e8-d4ebdf1ee9c2]
processed record [7] with id [9a843ed4-c097-469e-9af1-2f0fd2b5001d]
processed record [8] with id [4aed3228-4621-4e3f-b700-da59d52143e6]
processed record [9] with id [1839f701-3e12-4491-b191-201f9e6ba7e5]
processed record [10] with id [ee476c4d-3674-42be-9568-c348662e9310]
processed record [11] with id [c7a535e8-773e-4b95-8b65-903d682eb90c]
processed record [12] with id [b45e61ff-c5c1-468a-827c-2bab8aae78f6]
processed record [13] with id [12ebaf78-f1e3-4470-b707-b39ddd1fb719]
processed record [14] with id [85d070d6-3162-48e8-a8a8-97211fe04686]
processed record [15] with id [c1c70f8a-773

#### Step 2: Create Embeddings for each of the JSON Files

In [2]:
%pip install -q python-dotenv openai

Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

client = OpenAI()

response = client.embeddings.create(
    input="Hello world",
    model="text-embedding-3-small"
)

print(len(response.data[0].embedding))
print(response.data[0].embedding)

ImportError: cannot import name 'OpenAI' from 'openai' (/home/harit/.venv/openai3.12/lib/python3.11/site-packages/openai/__init__.py)

In [4]:
import os
import json

transformed_dir = "../data/transformed"
embedded_dir = "../data/embedded"

if not os.path.exists(embedded_dir):
    os.makedirs(embedded_dir)
    
def prepare_embedding_str(review_json):
    return f"REVIEW_TITLE: {review_json['review_title']} REVIEW_TEXT: {review_json['review_text']} HOTEL_NAME: {review_json['hotel_name']} HOTEL_CITY: {review_json['city']} HOTEL_STATE: {review_json['hotel_state']}"
    
client = OpenAI()
for file in os.listdir(transformed_dir):
    with open(f"{transformed_dir}/{file}", "r") as f:
        review = json.load(f)
        
        ## start here
        embedding_str = prepare_embedding_str(review)
        response = client.embeddings.create(
            input=embedding_str,
            model="text-embedding-3-small"
        )
        
        review['embedding'] = response.data[0].embedding
        
        with open(f"{embedded_dir}/{file}", "w") as f:
            json.dump(review, f, indent=2)